In [1]:
!pip install meshparty

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.3/94.3 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.8/704.8 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 543.2/543.2 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.8 MB/s eta 0:00:00


In [7]:
!pip install caveclient

In [28]:
!pip show meshparty

Name: meshparty
Version: 1.18.0
Summary: a service to work with meshes
Home-page: https://github.com/sdorkenw/MeshParty.git
Author: Sven Dorkenwald, Forrest Collman, Casey Schneider-Mizell
Author-email: svenmd@princeton.edu, forrestc@alleninstitute.org, caseys@alleninstitute.org
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: blosc, cloud-volume, dataclasses, h5py, imageio, multiwrapper, networkx, numpy, orjson, pandas, plyfile, pymeshfix, rtree, scikit-learn, scipy, shapely, tables, trimesh, vtk
Required-by: 


In [8]:
from caveclient import CAVEclient

In [12]:
import cloudvolume
from cloudvolume import CloudVolume

In [27]:
!pip show cloud_volume

Name: cloud-volume
Version: 11.0.2
Summary: A serverless client for reading and writing Neuroglancer Precomputed volumes both locally and on cloud services.
Home-page: https://github.com/seung-lab/cloud-volume/
Author: William Silversmith, Nico Kemnitz, Ignacio Tartavull, and others
Author-email: ws9@princeton.edu
License: License :: OSI Approved :: BSD License
Location: /usr/local/lib/python3.10/dist-packages
Requires: boto3, chardet, cloud-files, compressed-segmentation, DracoPy, fastremap, gevent, google-auth, google-cloud-core, google-cloud-storage, json5, jsonschema, microviewer, networkx, numpy, pathos, posix-ipc, protobuf, psutil, pysimdjson, python-dateutil, python-jsonschema-objects, requests, simplejpeg, six, tenacity, tqdm, urllib3
Required-by: meshparty


In [10]:
import pandas as pd
import numpy as np
import os
from meshparty import trimesh_io, trimesh_vtk, skeleton, utils
import vtk
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import time
from tqdm import tqdm

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from google.colab import userdata

os.environ["CAVECLIENT_TOKEN"] = userdata.get('CAVECLIENT_TOKEN')
caveclient_key = os.getenv('CAVECLIENT_TOKEN')

In [21]:
client = CAVEclient(auth_token=caveclient_key)

In [22]:
client.info.get_datastacks()

['minnie35_public_v0',
 'fanc_sandbox',
 'minnie65_public_v117',
 'minnie65_public_v343',
 'flywire_fafb_sandbox',
 'minnie65_public_v661',
 'pinky_sandbox',
 'minnie65_sandbox',
 'minnie65_public',
 'flywire_fafb_public']

In [24]:
datastack = 'minnie65_public'

In [14]:
# setup the mesh meta to handle downloads and caching
mesh_dir = '/content/drive/MyDrive/minnie65/meshes/'
seg_source = 'graphene://https://minnie.microns-daf.com/segmentation/table/minnie65_public' # static: "precomputed://gs://iarpa_microns/minnie/minnie65/seg"
# mm = trimesh_io.MeshMeta(cv_path=seg_source,
#                          disk_cache_path=mesh_dir,
#                          cache_size=20)

In [26]:
cv_obj = cloudvolume.CloudVolume(seg_source, use_https=True)

AuthenticationError: No Graphene authentication token was provided. Does ~/.cloudvolume/secrets/cave-secret.json exist?

In [ ]:
#client = CAVEclient("minnie65_public", auth_token=caveclient_key)
vol = CloudVolume(seg_source, secrets=caveclient_key)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
cell_id_list = [864691135135382809, 864691135135829529, 864691136228447569]


In [ ]:
start_time = time.time()

for i in tqdm(range(len(cell_id_list)), desc="Downloading Meshes"):
    mesh_start_time = time.time()

    downloadmesh = mm.mesh(seg_id=cell_id_list[i], remove_duplicate_vertices=True)

    mesh_elapsed_time = time.time() - mesh_start_time
    minutes, seconds = divmod(mesh_elapsed_time, 60)
    print(f"Processed mesh for cell_id {cell_id_list[i]} in {int(minutes)} minutes and {int(seconds)} seconds")

total_elapsed_time = time.time() - start_time
total_minutes, total_seconds = divmod(total_elapsed_time, 60)
print(f"Total processing time: {int(total_minutes)} minutes and {int(total_seconds)} seconds")

TypeError: MeshMeta.mesh() got an unexpected keyword argument 'secrets'

In [ ]:
import caveclient
client = caveclient.CAVEclient('minnie65_public')

In [ ]:
cell_id_list[0]

864691135135382809

In [ ]:
# workaround

import caveclient
client = caveclient.CAVEclient('minnie65_public', auth_token=caveclient_key)

cell_type_table = client.materialize.tables.aibs_metamodel_celltypes_v661(pt_root_id=cell_id_list[0]).query()
cell_type_table

,id_ref,created_ref,valid_ref,target_id,classification_system,cell_type,id,created,valid,volume,pt_supervoxel_id,pt_root_id,pt_position,bb_start_position,bb_end_position
0,43160,2023-12-19 22:48:55.688244+00:00,t,370517,excitatory_neuron,5P-ET,370517,2020-09-28 22:45:25.530497+00:00,t,568.771707,94876172508581974,864691135135382809,"[219120, 203312, 25375]","[nan, nan, nan]","[nan, nan, nan]"


In [ ]:
!pip show cloud_volume

Name: cloud-volume
Version: 8.33.0
Summary: A serverless client for reading and writing Neuroglancer Precomputed volumes both locally and on cloud services.
Home-page: https://github.com/seung-lab/cloud-volume/
Author: William Silversmith, Nico Kemnitz, Ignacio Tartavull, and others
Author-email: ws9@princeton.edu
License: License :: OSI Approved :: BSD License
Location: c:\users\drhan\anaconda3\envs\micronsbinder\lib\site-packages
Requires: boto3, chardet, cloud-files, compressed-segmentation, compresso, crackle-codec, DracoPy, fastremap, fpzip, gevent, google-auth, google-cloud-core, google-cloud-storage, json5, jsonschema, networkx, numpy, pathos, protobuf, psutil, pysimdjson, pyspng-seunglab, python-dateutil, python-jsonschema-objects, requests, simplejpeg, six, tenacity, tqdm, urllib3, zfpc
Required-by: imageryclient, meshparty


In [ ]:
# make a dictionary to hold the mesh file for each cell id in the list

mesh_dictionary = {}
for i in range(len(cell_id_list)):
    mesh_file = os.path.join(mesh_dir, str(cell_id_list[i]) + '_0' + '.h5')
    mesh_dictionary['cell_' + str(i)] = mm.mesh(filename = mesh_file)

In [ ]:
mesh_dictionary

In [ ]:
cell_gran = 1000
cell_opac = 1

fig, ax = plt.subplots()

for cellid in range(len(mesh_dictionary)):
    cell_key = 'cell_' + str(cellid)
    mesh = mesh_dictionary[cell_key]
    random_color = np.random.rand(3,)

    ax.scatter(
        mesh.vertices[::cell_gran, 0],
        mesh.vertices[::cell_gran, 1],
        s=1,
        alpha=cell_opac,
        color=random_color
    )

ax.invert_yaxis()  # Reverse the y-axis

plt.show()


In [ ]:
cell_gran = 1000
cell_opac = 1

fig = go.Figure()

for cellid in range(len(mesh_dictionary)):
    cell_key = 'cell_' + str(cellid)
    mesh = mesh_dictionary[cell_key]
    random_color = np.random.rand(3,)

    fig.add_trace(go.Scatter3d(
        x=mesh.vertices[::cell_gran, 0],
        y=mesh.vertices[::cell_gran, 1],
        z=mesh.vertices[::cell_gran, 2],
        mode='markers',
        marker=dict(size=1, opacity=cell_opac, color='rgb({}, {}, {})'.format(*random_color)),
        hoverinfo='none',
        showlegend=False
    ))

# Set the camera view
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0, y=0.5, z=1)  # Adjust these values to match the view in your screenshot
)

fig.update_layout(
    scene=dict(
        xaxis=dict(autorange='reversed'),  # Reverse the x-axis
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=0.4),  # Compress the z-axis
        camera=camera
    ),
     dragmode='orbit'  # Set default interaction mode to orbital rotation
)

fig.show()


Do not run the following if there are lot of meshes or large meshes (such as astrocytes)

In [ ]:
# opacity and color settings for cell membrane
cell_opac = 0.4
highlight_opac = 0.4
highlight_color = (0.2, 0.4, 0.7)

# cell membrane mesh

mesh_actor = {}
for i in range(len(mesh_dictionary)):
    random_color = list(np.random.random(size=3))
    mesh_actor['cell_' + str(i)] = trimesh_vtk.mesh_actor(mesh_dictionary['cell_'+str(i)], opacity=cell_opac, color=(random_color))

locals().update(mesh_actor)

# increase cell_i opacity to highlight it against other actors
cellnum = 0
mesh_actor['cell_' + str(cellnum)] = trimesh_vtk.mesh_actor(mesh_dictionary['cell_' + str(cellnum)], opacity=highlight_opac, color=(highlight_color))
locals().update(mesh_actor)

#creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(mesh_dictionary['cell_'+str(cellnum)].centroid, backoff=1500)

#render the actors, will open a pop up python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)

In [ ]:
# Convert each integer to a string
str_list = [str(i) for i in cell_id_list]

# Join the list into a single string with underscores
str_list_for_filename = "_".join(str_list)

# type(str_list_for_filename)

In [ ]:
# save image to disk
# uncomment to save

# camera = trimesh_vtk.oriented_camera(mesh.centroid, backoff=1500) # modify backoff accordingly

# trimesh_vtk.render_actors(mesh_actor.values(),
#                           filename='vtk_images/'+ str_list_for_filename +'.png',
#                           do_save=True,
#                           scale=6,
#                           camera=camera
#                          )